In [1]:
from binance.client import Client
from secrets import api_secrets, api_key
from binance.enums import *
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
from scipy import stats
from time import sleep
from IPython.display import clear_output
import winsound
import pandas as pd

client = Client(api_key, api_secrets)
print(client.get_account_status())

{'data': 'Normal'}


In [81]:
def p_delta(xf, xi):    
    delta_x = xf - xi
    porc_delta = (delta_x/xi)*100
    return porc_delta

def transf(brl, chz, xf):
    def calc_lucro(brl, chz, xf):    
        return chz*xf - brl
    l = calc_lucro(brl, chz, xf)    
    return l/(2-TX)

def od_compra(brl, chz, xf):
    t = transf(brl, chz, xf)
    t = abs(t)   
    brl_f = brl - t
    chz_f = chz + (t/xf)*(1-TX)
    return brl_f, chz_f, xf

def od_venda(brl, chz, xf):
    t = transf(brl, chz, xf)
    t = abs(t)  
    brl_f = brl + t*(1-TX)
    chz_f = chz - (t/xf)
    return brl_f, chz_f, xf

def remove_precos_vazios(todas_moedas, inter_tempo, tempo):
    def pega_precos(moeda):       
        klines = client.get_historical_klines(moeda, inter_tempo, tempo)
        precos = np.array([float(x[1]) for x in klines])
        return precos[1:-2], moeda
    xs = [pega_precos(x) for x in todas_moedas]
    xx = [x for x in xs if len(x[0]) is not 0]
    return xx

def dinheiro_inicial(M, TX, xii):
    brl_i = M/(2+TX)
    chz_i = (M-brl_i)*(1-TX)/xii
    return brl_i, chz_i

def pega_porcentagem(xizes, var, M, bloq):
    retorno = []
    for x in xizes:
        moeda = x[1]
        precos = x[0]
        xii = precos[0]
        xi = xii
        brl_i, chz_i = dinheiro_inicial(M, TX, xii)
        brl_f = brl_i
        chz_f = chz_i
        M1 = brl_i + chz_i*xii

        for xf in precos:
            p_delta_x = p_delta(xf, xi)  
            if abs(p_delta_x) > var:                
                t = transf(brl_f, chz_f, xf)
                              
                if t < - bloq:                                      
                    brl_f, chz_f, xi = od_compra(brl_f, chz_f, xf)                    
                elif t > bloq:                                      
                    brl_f, chz_f, xi = od_venda(brl_f, chz_f, xf)
                   
        
        M2 = brl_f + chz_f*xii*(1-TX)
        M3 = brl_f + chz_f*xf*(1-TX)
        p1 = 100*(M2 - M)/M
        p2 = 100*(M3 - M)/M
        print(M2)

        retorno.append([p1, p2, moeda])
    return retorno


In [10]:
#pegando todas moedas
t = client.get_ticker()
todas_moedas = [x['symbol'] for x in t if x['symbol'][-3:] == 'USD']
#Gana inicial
M = 50_000
TX = 0.001
#todas_moedas = ['LUNABUSD']

In [15]:
#xizes são os precos das diferentes moedas
xizes = remove_precos_vazios(todas_moedas[56:57], Client.KLINE_INTERVAL_1HOUR, "2 day ago UTC")
xizes



[(array([46.41, 46.16, 46.05, 46.32, 45.97, 46.  , 46.46, 46.74, 46.7 ,
         46.72, 46.9 , 47.42, 47.3 , 47.7 , 47.66, 47.37, 47.25, 47.36,
         47.61, 47.89, 47.41, 47.1 , 47.  , 46.97, 47.3 , 47.4 , 47.51,
         47.85, 49.09, 48.88, 48.78, 49.12, 50.21, 49.5 , 49.37, 49.53,
         49.59, 49.84, 49.87, 49.53, 49.69, 49.33, 49.85, 50.27, 50.01]),
  'DASHBUSD')]

In [84]:
var = 8
bloq = 10
M = 5000

xizes = [(np.array([1, 1.08]),'MEUMOEDA')]
porce = pega_porcentagem(xizes, var, M, bloq)
df = pd.DataFrame (porce, columns = ['p_xii', 'p_xf', 'moeda'])
df.sort_values(by=['p_xf'], ascending=False).head()

+
5002.399904448726


,p_xii,p_xf,moeda
0,0.047998,3.893999,MEUMOEDA


In [69]:
100*(46/50000)

0.092